Definicja struktur


In [1]:
abstract type GraphNode end
abstract type Operator <: GraphNode end

struct Constant{T} <: GraphNode
    output :: T
end

mutable struct Variable <: GraphNode
    output :: Any
    gradient :: Any
    name :: String
    Variable(output; name="?") = new(output, nothing, name)
end

mutable struct ScalarOperator{F} <: Operator
    inputs :: Any
    output :: Any
    gradient :: Any
    name :: String
    ScalarOperator(fun, inputs...; name="?") = new{typeof(fun)}(inputs, nothing, nothing, name)
end

mutable struct BroadcastedOperator{F} <: Operator
    inputs :: Any
    output :: Any
    gradient :: Any
    name :: String
    BroadcastedOperator(fun, inputs...; name="?") = new{typeof(fun)}(inputs, nothing, nothing, name)
end

Wizualizacja grafu

In [2]:
import Base: show, summary
show(io::IO, x::ScalarOperator{F}) where {F} = print(io, "op ", x.name, "(", F, ")");
show(io::IO, x::BroadcastedOperator{F}) where {F} = print(io, "op.", x.name, "(", F, ")");
show(io::IO, x::Constant) = print(io, "const ", x.output)
show(io::IO, x::Variable) = begin
    print(io, "var ", x.name);
    print(io, "\n ┣━ ^ "); summary(io, x.output)
    print(io, "\n ┗━ ∇ ");  summary(io, x.gradient)
end

show (generic function with 282 methods)

Budowa grafu


In [3]:
function visit(node::GraphNode, visited, order)
    if node ∈ visited
    else
        push!(visited, node)
        push!(order, node)
    end
    return nothing
end
    
function visit(node::Operator, visited, order)
    if node ∈ visited
    else
        push!(visited, node)
        for input in node.inputs
            visit(input, visited, order)
        end
        push!(order, node)
    end
    return nothing
end

function topological_sort(head::GraphNode)
    visited = Set()
    order = Vector()
    visit(head, visited, order)
    return order
end

topological_sort (generic function with 1 method)

Forward Pass

In [4]:
reset!(node::Constant) = nothing
reset!(node::Variable) = node.gradient = nothing
reset!(node::Operator) = node.gradient = nothing

compute!(node::Constant) = nothing
compute!(node::Variable) = nothing
compute!(node::Operator) =
    node.output = forward(node, [input.output for input in node.inputs]...)

function forward!(order::Vector)
    for node in order
        compute!(node)
        reset!(node)
    end
    return last(order).output
end

forward! (generic function with 1 method)

Backward pass

In [5]:
update!(node::Constant, gradient) = nothing
update!(node::GraphNode, gradient) = if isnothing(node.gradient)
    node.gradient = gradient else node.gradient .+= gradient
end

function backward!(order::Vector; seed=1.0)
    result = last(order)
    result.gradient = seed
    @assert length(result.output) == 1 "Gradient is defined only for scalar functions"
    for node in reverse(order)
        backward!(node)
    end
    return nothing
end

function backward!(node::Constant) end
function backward!(node::Variable) end
function backward!(node::Operator)
    inputs = node.inputs
    gradients = backward(node, [input.output for input in inputs]..., node.gradient)
    for (input, gradient) in zip(inputs, gradients)
        update!(input, gradient)
    end
    return nothing
end

backward! (generic function with 4 methods)

Zaimplementowane operacje skalarne

In [6]:
import Base: ^
^(x::GraphNode, n::GraphNode) = ScalarOperator(^, x, n)
forward(::ScalarOperator{typeof(^)}, x, n) = return x^n
backward(::ScalarOperator{typeof(^)}, x, n, g) = tuple(g * n * x ^ (n-1), g * log(abs(x)) * x ^ n)

import Base: sin
sin(x::GraphNode) = ScalarOperator(sin, x)
forward(::ScalarOperator{typeof(sin)}, x) = return sin(x)
backward(::ScalarOperator{typeof(sin)}, x, g) = tuple(g * cos(x))

backward (generic function with 2 methods)

Operacje macierzowe

In [7]:
import Base: *
import LinearAlgebra: mul!, diagm
# x * y (aka matrix multiplication)
*(A::GraphNode, x::GraphNode) = BroadcastedOperator(mul!, A, x)
forward(::BroadcastedOperator{typeof(mul!)}, A, x) = return A * x
backward(::BroadcastedOperator{typeof(mul!)}, A, x, g) = tuple(g * x', A' * g)

# x .* y (element-wise multiplication)
Base.Broadcast.broadcasted(*, x::GraphNode, y::GraphNode) = BroadcastedOperator(*, x, y)
forward(::BroadcastedOperator{typeof(*)}, x, y) = return x .* y
backward(node::BroadcastedOperator{typeof(*)}, x, y, g) = let
    𝟏 = ones(length(node.output))
    Jx = diagm(y .* 𝟏)
    Jy = diagm(x .* 𝟏)
    tuple(Jx' * g, Jy' * g)
end

Base.Broadcast.broadcasted(-, x::GraphNode, y::GraphNode) = BroadcastedOperator(-, x, y)
forward(::BroadcastedOperator{typeof(-)}, x, y) = return x .- y
backward(::BroadcastedOperator{typeof(-)}, x, y, g) = tuple(g,-g)

Base.Broadcast.broadcasted(+, x::GraphNode, y::GraphNode) = BroadcastedOperator(+, x, y)
forward(::BroadcastedOperator{typeof(+)}, x, y) = return x .+ y
backward(::BroadcastedOperator{typeof(+)}, x, y, g) = tuple(g, g)

import Base: sum
sum(x::GraphNode) = BroadcastedOperator(sum, x)
forward(::BroadcastedOperator{typeof(sum)}, x) = return sum(x)
backward(::BroadcastedOperator{typeof(sum)}, x, g) = let
    𝟏 = ones(length(x))
    J = 𝟏'
    tuple(J' * g)
end

Base.Broadcast.broadcasted(/, x::GraphNode, y::GraphNode) = BroadcastedOperator(/, x, y)
forward(::BroadcastedOperator{typeof(/)}, x, y) = return x ./ y
backward(node::BroadcastedOperator{typeof(/)}, x, y::Real, g) = let
    𝟏 = ones(length(node.output))
    Jx = diagm(𝟏 ./ y)
    Jy = (-x ./ y .^2)
    tuple(Jx' * g, Jy' * g)
end

import Base: max
Base.Broadcast.broadcasted(max, x::GraphNode, y::GraphNode) = BroadcastedOperator(max, x, y)
forward(::BroadcastedOperator{typeof(max)}, x, y) = return max.(x, y)
backward(::BroadcastedOperator{typeof(max)}, x, y, g) = let
    Jx = diagm(isless.(y, x))
    Jy = diagm(isless.(x, y))
    tuple(Jx' * g, Jy' * g)
end

backward (generic function with 9 methods)

Wczytanie danych

In [8]:
using MLDatasets

# load training set
train_x, train_y = MNIST.(split=:train)[:]

# load test set
test_x,  test_y  = MNIST.(split=:test)[:]


(features = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; … ;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], targets = [7, 2, 1, 0, 4, 1, 4, 9, 5, 9  …  7, 8, 9, 0, 1, 2, 3, 4, 5, 6])

Dopracowanie formatu danych

In [9]:
function onehotencode(y::AbstractVector{T}) where T<:Integer
    # Znajdujemy unikalne klasy w wektorze `y`
    classes = sort(unique(y))
    # Liczba klas
    num_classes = length(classes)
    # Liczba próbek
    num_samples = length(y)
    # Inicjujemy macierz one-hot encoding `y`
    y_onehot = zeros(T, num_classes, num_samples)
    # Iterujemy po próbkach i ustawiamy wartości w macierzy one-hot encoding
    for i = 1:num_samples
        # Znajdujemy indeks klasy odpowiadającej elementowi `y[i]`
        index = findfirst(classes .== y[i])
        # Ustawiamy 1 w odpowiednim miejscu w macierzy
        for j = 1:num_classes
            if j == index
                y_onehot[j, i] = 1
            end
        end
    end
    # Zwracamy macierz one-hot encoding `y`
    return y_onehot
end

train_y_onehot = onehotencode(train_y)
test_y_onehot = onehotencode(test_y)


10×10000 Matrix{Int64}:
 0  0  0  1  0  0  0  0  0  0  1  0  0  …  0  0  0  0  0  1  0  0  0  0  0  0
 0  0  1  0  0  1  0  0  0  0  0  0  0     0  0  0  0  0  0  1  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  1  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  1  0  0  0
 0  0  0  0  1  0  1  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  1  0  0  0  0  …  1  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  0  0  1  0     0  1  0  0  0  0  0  0  0  0  0  1
 1  0  0  0  0  0  0  0  0  0  0  0  0     0  0  1  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  1  0  0  1     0  0  0  0  1  0  0  0  0  0  0  0

Konwolucja

In [10]:
conv(x::GraphNode, f::GraphNode) = BroadcastedOperator(conv, x, f)
forward(::BroadcastedOperator{typeof(conv)}, x, f) = let 
# rozmiary danych wejściowych
m, n, c = size(x)
p, q, ci, k = size(f)

# wynikowa macierz konwolucji
output = zeros(m-p+1, n-q+1, k)

# pętla po każdym filtrowaniu
for l = 1:k
    # pętla po każdym kanałach
    for d = 1:c
        # pętla po każdym rzędzie obrazu wejściowego
        for i = 1:m-p+1
            # pętla po każdej kolumnie obrazu wejściowego
            for j = 1:n-q+1
                # wartość pojedynczego elementu wynikowego
                val = 0.0
                # pętla po każdym elemencie w fragmencie obrazu
                for s = 1:p
                    for t = 1:q
                        # iloczyn skalarny
                        val += x[i+s-1, j+t-1, d] .* f[s, t, d, l]
                    end
                end
                # przypisanie wartości do macierzy wynikowej
                output[i, j, l] += val
            end
        end
    end
end

output
end
backward(node::BroadcastedOperator{typeof(conv)}, x, f, g) = let
    # rozmiary danych wejściowych
    m, n, c = size(x)
    p, q, ci, k = size(f)
    # macierz gradientu wag
    f_gradient = zeros(size(f))
    # macierz gradientu wejścia
    x_gradient = zeros(size(x))
    # pętla po każdym filtrowaniu
    for l = 1:k
        # pętla po każdym kanałach
        for d = 1:c
            # pętla po każdym rzędzie obrazu wejściowego
            for i = 1:m-p+1
                # pętla po każdej kolumnie obrazu wejściowego
                for j = 1:n-q+1
                    # gradient funkcji kosztu względem pojedynczego elementu z wyjścia
                    output_grad = g[i, j, l]
                    # pętla po każdym elemencie w fragmencie obrazu
                    for s = 1:p
                        for t = 1:q
                            # gradient funkcji kosztu względem pojedynczego elementu wagi
                            f_gradient[s, t, d, l] += x[i+s-1, j+t-1, d] * output_grad
                            # gradient funkcji kosztu względem pojedynczego elementu wejścia
                            x_gradient[i+s-1, j+t-1, d] += f[s, t, d, l] * output_grad
                        end
                    end
                end
            end
        end
    end
    tuple(x_gradient, f_gradient)
end


backward (generic function with 10 methods)

Funkcja aktywacji ReLu

In [11]:
relu(x::GraphNode) = BroadcastedOperator(relu, x)
forward(::BroadcastedOperator{typeof(relu)}, x) =return max.(0, x)
backward(node::BroadcastedOperator{typeof(relu)}, x, g) = let
    y = node.output
    tuple(g .* (y .> 0))
end

backward (generic function with 11 methods)

MaxPooling

In [12]:
maxpooling(x::GraphNode) = BroadcastedOperator(maxpooling, x)
forward(::BroadcastedOperator{typeof(maxpooling)}, x) = let
# rozmiary danych wejściowych
m, n, c = size(x)
ps = 2
qs = 2

# rozmiary danych wyjściowych
out_m, out_n = div(m, ps), div(n, qs)

# wynikowa macierz maxpoolingu
output = zeros(Float64, out_m, out_n, c)

# pętla po każdym kanale
for d = 1:c
    # pętla po każdym rzędzie wynikowej macierzy
    for i = 1:out_m
        # pętla po każdej kolumnie wynikowej macierzy
        for j = 1:out_n
            # indeksy fragmentu macierzy wejściowej do przetworzenia
            i_start = (i-1)*ps + 1
            j_start = (j-1)*qs + 1
            i_end = min(i_start+ps-1, m)
            j_end = min(j_start+qs-1, n)
            # fragment macierzy wejściowej do przetworzenia
            x_fragment = x[i_start:i_end, j_start:j_end, d]
            # maksymalna wartość w fragmencie
            max_value, max_index = findmax(x_fragment)
            # obliczenie pozycji elementu o najwyższej wartości
            max_position = LinearIndices(x_fragment)[max_index]
            # obliczenie pozycji elementu w macierzy wyjściowej
            i_position = i_start + div(max_position-1, ps)
            j_position = j_start + mod(max_position-1, qs)
            # przypisanie wartości do elementu w macierzy wyjściowej
            output[i, j, d] = max_value
        end
    end
end
output     
end
backward(node::BroadcastedOperator{typeof(maxpooling)}, x, g) = let
# rozmiary danych wejściowych
m, n, c = size(x)
ps = 2
qs = 2
# rozmiary danych wyjściowych
out_m, out_n = div(m, ps), div(n, qs)
# macierz do przechowywania gradientów dla każdego elementu wejściowego
grad_input = zeros(Float64, m, n, c)
# pętla po każdym kanale
for d = 1:c
    # pętla po każdym rzędzie wynikowej macierzy
    for i = 1:out_m
        # pętla po każdej kolumnie wynikowej macierzy
        for j = 1:out_n
            # indeksy fragmentu macierzy wejściowej do przetworzenia
            i_start = (i-1)*ps + 1
            j_start = (j-1)*qs + 1
            i_end = min(i_start+ps-1, m)
            j_end = min(j_start+qs-1, n)
            # fragment macierzy wejściowej do przetworzenia
            x_fragment = x[i_start:i_end, j_start:j_end, d]
            # maksymalna wartość w fragmencie
            max_value, max_index = findmax(x_fragment)
            # obliczenie pozycji elementu o najwyższej wartości
            max_position = LinearIndices(x_fragment)[max_index]
            # obliczenie pozycji elementu w macierzy wejściowej
            i_position = i_start + div(max_position-1, ps)
            j_position = j_start + mod(max_position-1, qs)
            # przekazanie gradientu do elementu, który przyczynił się do wyznaczenia maksymalnej wartości
            grad_input[i_position, j_position, d] += g[i, j, d]
        end
    end
end

grad_input
end

backward (generic function with 12 methods)

Warstwa Flatten

In [13]:
flatten(x::GraphNode) = BroadcastedOperator(flatten, x)
forward(::BroadcastedOperator{typeof(flatten)}, x) = return reshape(x, :)
backward(node::BroadcastedOperator{typeof(flatten)}, x, g) = (reshape(g, size(x)),)

backward (generic function with 13 methods)

Warstwa Dense

In [14]:
dense(x::GraphNode,weight::GraphNode) = BroadcastedOperator(dense, x, weight)
forward(::BroadcastedOperator{typeof(dense)}, x, weight) = return weight * x
backward(node::BroadcastedOperator{typeof(dense)}, x, weight, g) = let 
    # obliczenie pochodnej względem x
    input_gradient = weight' * g
    # obliczenie pochodnej względem wag
    weight_gradient = g * x'
    tuple(input_gradient, weight_gradient)
end

backward (generic function with 14 methods)

Funkcja aktywacji softmax

In [15]:
softmax(x::GraphNode) = BroadcastedOperator(softmax, x)
forward(::BroadcastedOperator{typeof(softmax)}, x) = return exp.(x) ./ sum(exp.(x), dims=1)
backward(node::BroadcastedOperator{typeof(softmax)}, x, g) = let
    y = node.output
    #obliczenie jacobjanu
    J = diagm(vec(y)) .- y * y'
    #zwrócenie gradientu wejściowego
    tuple(J' * g)
end

backward (generic function with 15 methods)

Obliczanie entropi krzyżowej

In [16]:
cross_entropy(y_pred::GraphNode, y_true::GraphNode) = BroadcastedOperator(cross_entropy, y_pred, y_true)
forward(::BroadcastedOperator{typeof(cross_entropy)}, y_pred, y_true) = let
    m = size(y_pred, 2)
    # obliczenie wartości funkcji straty
    loss = -1/m * sum(y_true .* log.(y_pred))
    loss
end
backward(node::BroadcastedOperator{typeof(cross_entropy)}, y_pred, y_true, g) = let
    m = size(y_pred, 2)
    # obliczenie gradientu po y_pred
    g_y = -1/m * (y_true ./ y_pred)
    # zwrócenie gradientu funkcji kosztu
    tuple(g .* g_y)
end

backward (generic function with 16 methods)

Inicjacja wag

In [17]:
using Random
function create_filter(dims::Tuple, min_val::Float64, max_val::Float64)
    # tworzymy macierz wypełnioną zerami
    matrix = zeros(dims)

    # losujemy liczby i przypisujemy do macierzy
    for i in 1:dims[1]
        for j in 1:dims[2]
            for k in 1:dims[3]
                for l in 1:dims[4]
                    matrix[i, j, k, l] = rand() * (max_val - min_val) + min_val
                end
            end
        end
    end

    return matrix
end

function create_weights(dims::Tuple, min_val::Float64, max_val::Float64)
    # tworzymy macierz wypełnioną zerami
    matrix = zeros(dims)

    # losujemy liczby i przypisujemy do macierzy
    for i in eachindex(matrix)
        matrix[i] = rand() * (max_val - min_val) + min_val
    end

    return matrix
end

create_weights (generic function with 1 method)

Inicjacja pierwszej sieci

In [18]:
y_target=Variable(train_y_onehot[:,1])

x = Variable((reshape(train_x[:,:,1], (28,28, 1))), name="input") 
x.output = reshape(train_x[:,:,1], (28,28, 1))

filters = Variable(create_filter((3,3,1,4),-0.05, 0.05))
output1 = conv(x,filters)
output2 = relu(output1)
output3 = maxpooling(output2)
output4 = flatten(output3)
weights = Variable(create_weights((10,676),-0.05, 0.05))
output5 = dense(output4,weights)
output6 = softmax(output5)
output7 = cross_entropy(output6,y_target)
graph = topological_sort(output7)
forward!(graph)   
backward!(graph)

Uczenie

In [19]:
using Printf

#Liczba epok
const num_epochs = 10

# Stawiamy współczynnik uczenia jako zmienną, aby łatwo go zmieniać
const learning_rate = 0.001

# Funkcja do ewaluacji modelu
function evaluate_model(graph, test_x, test_y_onehot, test_y)
    correct = 0
    for i = 1:length(test_y)
        # Przekształć dwuwymiarowe dane obrazu na trójwymiarowy tensor
        x.output = reshape(test_x[:,:,i], (28, 28, 1))
        # Ustaw docelowe wyjście sieci dla tego przykładu
        y_target.output = test_y_onehot[:,i]  
        # Oblicz wyjście sieci neuronowej
        forward!(graph)
        # Jeśli przewidziana cyfra zgadza się z rzeczywistą, zwiększ licznik poprawnych odpowiedzi
        if y_target.output[argmax(output6.output)] == 1
            correct = correct + 1 
        end
    end
    return correct/length(test_y)
end

# Pętla ucząca
for e = 1:num_epochs
    for i = 1:length(train_y)
        # Przekształć dwuwymiarowe dane obrazu na trójwymiarowy tensor
        x.output = reshape(train_x[:,:,i], (28,28,1))
        # Ustaw docelowe wyjście sieci dla tego przykładu
        y_target.output = train_y_onehot[:,i]  
        # Oblicz wyjście sieci neuronowej
        forward!(graph)
        # Jeśli wyjście sieci neuronowej jest NaN, przerwij pętlę
        if isnan(output7.output)
            println("Wartość nan na wyjściu!")
            break 
        end
        # Oblicz gradienty funkcji straty i aktualizuj wagi sieci
        backward!(graph)
        filters.output = filters.output - filters.gradient * learning_rate
        weights.output = weights.output - weights.gradient * learning_rate
    end
    
    # Policz dokładność modelu na zbiorze testowym
    accuracy = evaluate_model(graph, test_x, test_y_onehot, test_y)
    # Wydrukuj dokładność modelu na zbiorze testowym
    @printf("Epoch: %d, Accuracy: %.4f\n", e, accuracy)
end

Epoch: 1, Accuracy: 0.6201
Epoch: 2, Accuracy: 0.6755


Epoch: 3, Accuracy: 0.6799


Epoch: 4, Accuracy: 0.6681


Epoch: 5, Accuracy: 0.6698
Epoch: 6, Accuracy: 0.6817


Epoch: 7, Accuracy: 0.6899
Epoch: 8, Accuracy: 0.6993


Epoch: 9, Accuracy: 0.7036
Epoch: 10, Accuracy: 0.7073
